# Imports/Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

In [2]:
path_train = '/kaggle/input/asl-signs/train.csv'

In [3]:
dftr = pd.read_csv(path_train)

We select the first associated landmark files to get a feel for how the data feels.

In [4]:
dftr

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [5]:
example_landmark = pd.read_parquet('/kaggle/input/asl-signs/train_landmark_files/26734/1000035562.parquet')

It looks like the left hand has no information.
The data later on should reveal that the pose will have the left hand out of screen.

In [6]:
example_landmark[example_landmark['type'] == 'left_hand'].notnull()['x'] == True

468      False
469      False
470      False
471      False
472      False
         ...  
12430    False
12431    False
12432    False
12433    False
12434    False
Name: x, Length: 483, dtype: bool

In [7]:
sgl_in = example_landmark[example_landmark['frame'] == 20]

In [8]:
sgl_in['type'].value_counts()

face          468
pose           33
left_hand      21
right_hand     21
Name: type, dtype: int64

In [9]:
sgl_in_rh = sgl_in[sgl_in['type'] == 'right_hand']

In [10]:
sgl_in_rh

,frame,row_id,type,landmark_index,x,y,z
522,20,20-right_hand-0,right_hand,0,0.317082,0.576334,2.273533e-07
523,20,20-right_hand-1,right_hand,1,0.378220,0.522858,2.524794e-02
524,20,20-right_hand-2,right_hand,2,0.400223,0.473187,3.187834e-02
525,20,20-right_hand-3,right_hand,3,0.422157,0.436533,2.407637e-02
526,20,20-right_hand-4,right_hand,4,0.446464,0.403041,1.425959e-02
527,20,20-right_hand-5,right_hand,5,0.350601,0.440231,6.016464e-02
528,20,20-right_hand-6,right_hand,6,0.376617,0.397908,4.689403e-02
529,20,20-right_hand-7,right_hand,7,0.415079,0.385037,2.567031e-02
530,20,20-right_hand-8,right_hand,8,0.447915,0.381096,1.004762e-02
531,20,20-right_hand-9,right_hand,9,0.327356,0.440892,2.936660e-02


In [11]:
def add_init_c(start, end, hand):
    return (
        pd.concat([hand['x'][start:start+1], hand['x'][end[0]:end[1]]]), 
        pd.concat([hand['y'][start:start+1], hand['y'][end[0]:end[1]]]), 
        pd.concat([hand['z'][start:start+1], hand['z'][end[0]:end[1]]])
        )

In [12]:
def plot_hand(hand, td):
    fig, ax = plt.subplots()
    fig.set_size_inches(6, 6)

    if td:
        ax = Axes3D(fig, auto_add_to_figure=False)
        fig.add_axes(ax)

    ind = [[0, [0, 5]], [0, [5, 9]], [0, [17, 21]], [5, [9, 13]], [17, [13, 17]], [9, [13, 14]]]

    for i, k in ind: 
        x, y, z = add_init_c(i, k, hand)
        if td:
            ax.plot(x, -1*y, z)
        else:
            ax.plot(x, -1*y)

In [13]:
plot_hand(sgl_in_rh, True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_hand(sgl_in_rh, False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
sgl_in_lh = sgl_in[sgl_in['type'] == 'left_hand']

In [16]:
sgl_in_lh

,frame,row_id,type,landmark_index,x,y,z
468,20,20-left_hand-0,left_hand,0,NaN,NaN,NaN
469,20,20-left_hand-1,left_hand,1,NaN,NaN,NaN
470,20,20-left_hand-2,left_hand,2,NaN,NaN,NaN
471,20,20-left_hand-3,left_hand,3,NaN,NaN,NaN
472,20,20-left_hand-4,left_hand,4,NaN,NaN,NaN
473,20,20-left_hand-5,left_hand,5,NaN,NaN,NaN
474,20,20-left_hand-6,left_hand,6,NaN,NaN,NaN
475,20,20-left_hand-7,left_hand,7,NaN,NaN,NaN
476,20,20-left_hand-8,left_hand,8,NaN,NaN,NaN
477,20,20-left_hand-9,left_hand,9,NaN,NaN,NaN


In [17]:
sgl_in_p = sgl_in[sgl_in['type'] == 'pose']

In [18]:
sgl_in_p

,frame,row_id,type,landmark_index,x,y,z
489,20,20-pose-0,pose,0,0.524597,0.329134,-0.976228
490,20,20-pose-1,pose,1,0.566472,0.284107,-0.913417
491,20,20-pose-2,pose,2,0.590944,0.285315,-0.913624
492,20,20-pose-3,pose,3,0.612991,0.287311,-0.913853
493,20,20-pose-4,pose,4,0.493541,0.286207,-0.892349
494,20,20-pose-5,pose,5,0.473928,0.288163,-0.891821
495,20,20-pose-6,pose,6,0.454607,0.290489,-0.892151
496,20,20-pose-7,pose,7,0.646151,0.311624,-0.477587
497,20,20-pose-8,pose,8,0.438918,0.312438,-0.347345
498,20,20-pose-9,pose,9,0.573214,0.374347,-0.816377


In [19]:
fig, ax = plt.subplots()
fig.set_size_inches(6, 6)

s = [20] * sgl_in_p.shape[0]
s

ax.scatter(sgl_in_p['x'], -1*sgl_in_p['y'], s=s)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …